### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [1]:
import json
import warnings

PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (420, 297),
    'A4': (297, 210),
    'A5': (210, 148),
    'A6': (148, 105)
}
class ReadableEntity:
    """
    Basic properties for any readable object
    """
    def __init__(self, title, author, genre, pages, page_format, rareness, content):
        self.book_title = title
        self.author = author
        self.genre = genre
        self.pages = pages
        self.page_format = page_format
        self.rareness = rareness
        self.content = content 
        if self.page_format in PAGES_FORMAT:     
            self.page_size = PAGES_FORMAT[page_format]
        else:
            warnings.warn('No such page format in dictionary, default A5 page size is used', Warning)
            self.page_size = PAGES_FORMAT['A5']
         
    
    @property
    def pages_area(self):        
        return self.page_size[0] * self.page_size[1] * self.pages

class Journal(ReadableEntity):
    """
    Has all basic readable objects features plus the ones specific for journals like volume, language, year
    """
    def __init__(self,  title, author, genre, pages, page_format, rareness, content, volume, year, language):
        ReadableEntity.__init__(self, title, genre, pages, page_format, rareness, content)
        self.volume = volume
        self.year = year
        self.language = language


class Book(ReadableEntity):
    """
    Has all basic readable objects features plus the ones specific for journals like edition, language, publisher, year
    """
    def __init__(self,  title, author, genre, pages, page_format, rareness, content, year,publisher, edition, language):
        ReadableEntity.__init__(self, title,author, genre, pages, page_format, rareness, content)       
        self.year = year
        self.publisher = publisher
        self.edition = edition
        self.language = language


class Exporter:   
    def __init__(self, rareness, path):        
        if rareness > 8:
            self.dump_json(path)
            
    def dump_json(self, path):        
        with open(path, 'w') as f:            
            json.dump(self.__dict__, f)
            
class LibraryAttribtutes:  
    """
    Library object is present is some quantity and can be given to people or be returned.
    """
    def __init__(self, shelf, quantity):   
        self.quantity = quantity
        self.shelf = shelf    
    
    def giveout(self):
        if self.quantity > 0:
            self.quantity -= 1
            return self
        else: 
            raise Exception("All book's copies are in hand")

    def takeback(self):
        self.quantity += 1 
        return self
        
class LibraryJournal(Journal, Exporter, LibraryAttribtutes):
    def __init__(self, title,author, genre, pages, page_format, rareness, content, volume, year, language,
                store_loc,shelf, quantity = 1):
        Journal.__init__(self, title,author, genre, pages, page_format, rareness, content, volume, year, language)        
        
        Exporter.__init__(self, rareness, store_loc)
        LibraryAttribtutes.__init__(self, shelf, quantity)

class LibraryBook(Book, Exporter, LibraryAttribtutes):
    def __init__(self, title,author, genre, pages, page_format,
                 rareness, content, year, publisher, edition,
                 language, store_loc, shelf, quantity = 1):
        Book.__init__(self, title,author, genre, pages, page_format, rareness, content, year, publisher, edition, language) 
        Exporter.__init__(self, rareness, store_loc)
        LibraryAttribtutes.__init__(self, shelf, quantity)
        
   

In [2]:
title = 'Lorem Ipsum'
author = 'Ipsum Lorem'
genre = 'loresius ipsumius'
pages = 491
rareness = 9
content = 'Lorep Ipsum' * pages
page_format = 'A5'
year = 1535
publisher = "Gutenberg Wiley and O'Reily"
edition = '2nd'
language = 'ENG'
quantity = 1
store_loc = f"{title}, {author}, {edition} backup"
shelf = 'B16'


In [3]:
book = LibraryBook(title,author, genre, pages, page_format,
                 rareness, content, year, publisher, edition,
                 language, store_loc, shelf, quantity = 1 )
print(book.quantity)
book.giveout()
print(book.quantity)
book.takeback()
print(book.quantity)

1
0
1


In [4]:
book.page_size, book.pages_area

((210, 148), 15260280)

### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


In [5]:
import sys
from gc import get_referents

def getrealsize(x):
    """
    Returns real size of object (excl. type objects)
    """
    total_size, already_checked, objects_to_check = 0, set(), []
    objects_to_check.append(x)
    while len(objects_to_check) > 0:
        for _object in objects_to_check:
            if id(_object) in already_checked or isinstance(_object, type):   
                objects_to_check.remove(_object)
            else:
                total_size += sys.getsizeof(_object)
                already_checked.add(id(_object))
                objects_to_check += get_referents(_object)   
                objects_to_check.remove(_object)
    return total_size

In [6]:
getrealsize(['aaaaaaa']), getrealsize('aaaaaaa'), getrealsize([])

(128, 56, 64)

In [7]:
getrealsize('a'), getrealsize('aa'), getrealsize('aaa')

(50, 51, 52)

### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)**

5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

In [8]:
# import operator
from cmath import exp
from math import pi

        
class Polynomial:
    
    def __init__(self, *coefficients):
        """ input: coefficients are in the form a_0, a_1,... a_n 
        """
        self.coefficients = list(coefficients) 
            
    @property
    def degree(self):
        return len(self.coefficients)
    
    def normalize(self, degree):
        while len(self.coefficients) < degree:
            self.coefficients.append(0)                      
    
    def __add__(self, another):           

        max_degree = max(self.degree, another.degree)
        self.normalize(max_degree)
        another.normalize(max_degree)

        new_coefs = [x + y for x, y in zip(another.coefficients, self.coefficients)] 
        return Polynomial(*new_coefs)
    
    def __neg__(self):
        "Return -self"
        return Polynomial(*[-i for i in self.coefficients])
    
    def __eq__(self, other):
        
        if isinstance(other, Polynomial):
            x = [round(i,4) for i in self.coefficients]
            y = [round(i,4) for i in other.coefficients]
            return x == y
        else:
            return self.coefficients == [other]
    
    
    def __sub__(self, another):                         
        return -another + self          
    
    def __mul__(self, another):
        """ 
        Just coefficient-wise multiplication of O(n^2) complexity
        """            

        coeffs = {i:0 for i in range(len(self.coefficients)+ len(another.coefficients))}  

        for index, ivalue in enumerate(self.coefficients):
            for jndex, jvalue in enumerate(another.coefficients):
                coeffs[index+jndex] += ivalue*jvalue

        new_coefs = list(coeffs.values())
        try:
            while new_coefs[-1] == 0:
                new_coefs.pop()
            return Polynomial(*new_coefs) 
        except:
            return Polynomial(0) 

       
    def fast_multiply(self, other): 
        """
        Uses FFT multiplication under the hood
        """
        def FFT(A):
            N = len(A)
            if len(A) <= 1:
                return [sum(A)]

            part1 = FFT(A[0::2])
            part2 = FFT(A[1::2])
            T = [exp(-2j*pi*k/N)*part2[k] for k in range(N//2)]
            return [part1[k] + T[k] for k in range(N//2)] + [part1[k] - T[k] for k in range(N//2)]
        

        A = self.coefficients
        B = other.coefficients
        n = 2**(len(A)+len(B)-2).bit_length()

        while len(A) < n: A.append(0)
        while len(B) < n: B.append(0)

        AT = FFT(A)
        BT = FFT(B)
        C = [AT[i]*BT[i] for i in range(len(AT))]
        new_C =[i.imag+1j*i.real for i in C]

        D = [round(a.imag/len(C)) for a in FFT(new_C)]
        while len(D)>1 and D[-1] == 0: D.pop()
        return Polynomial(*D)    
    
    def power(self, n):
        """
        Uses regular multiplication under the hood
        """
        if n < 0: 
            raise ValueError('Cannot perform a negative degree power')
        result, i  = Polynomial(1), 0
        while i < n:
            result =self*result
            i += 1
        return result
    
    def fast_power(self, n):
        if n < 0: 
            raise ValueError('Cannot perform a negative degree power')
        result, i  = Polynomial(1), 0
        while i < n:
            result = self.fast_multiply(result) 
            i += 1
        return result        
       
    @property    
    def derivative(self):                        
        new_coeffs = [self.coefficients[i+1]*(i+1) for i in range(len(self.coefficients[1:]))]        
        return Polynomial(*new_coeffs)            
    
    @property
    def integrate(self):                
        new_coeffs = [self.coefficients[i]/(i+1) for i in range(len(self.coefficients))]
        new_coeffs.insert(0, 0)    
        return Polynomial(*new_coeffs)            
               
    def divide(self, other):
        while other.coefficients[-1] == 0:
            other.coefficients.pop()            
        first_coef = other.coefficients[-1]
        
        result = self.coefficients[::-1]        
        for i in range(self.degree - other.degree + 1):
            result[i] /= first_coef                                      
            new_coef = result[i]
            if new_coef != 0: 
                for j in range(1, len(other.coefficients)):                      
                    result[i + j] -= new_coef * other.coefficients[other.degree - j - 1]
        rem, res = Polynomial(*result[::-1][other.degree - 1:]), Polynomial(*result[::-1][:other.degree - 1])  
        return rem, res
    
    
    def __call__(self, x):    
        res = 0
        for index, coeff in enumerate(self.coefficients):
            res += coeff * x**index
        return res         
    
    def __repr__(self):
        """
        Method to return the canonical string representation 
        of a polynomial.
        """ 
        interm = "+".join([f"{value}*x^{i}" for i, value in  enumerate(self.coefficients) if value != 0])
        interm = "+".join(interm.split('+')[::-1])
        return  interm.replace("+-",'-').replace("*x^0",'')

In [9]:
a = Polynomial(1,4,6,-7,-0,-9,0,9)
b = Polynomial(1,1,0,5,0,2,0,1)
rs = a.divide(b)
assert rs[0]*b + rs[1] == a

In [10]:
a = Polynomial(1,4,6,8, 32,231,13,211,221)
b = Polynomial(1,4,6,8,31,13,211)
rs = a.divide(b)
assert rs[0]*b + rs[1] == a

In [11]:
assert a.fast_power(5) == a.power(5)

In [12]:
assert a*b == a.fast_multiply(b)

In [13]:
a = Polynomial(-4,+40,0,16,-70,-0,-9,0,9)
a, a.integrate, a.derivative, a(10)

(9*x^8-9*x^6-70*x^4+16*x^3+40*x^1-4,
 1.0*x^9-1.2857142857142858*x^7-14.0*x^5+4.0*x^4+20.0*x^2-4.0*x^1,
 72*x^7-54*x^5-280*x^3+48*x^2+40,
 890316396)

### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

#https://late.am/post/2012/06/18/what-the-heck-is-an-xrange

In [14]:
from math import ceil
from collections import Sequence, Iterator

/Users/ayunts/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


In [15]:
class custom_range:
    def __init__(self, *args):
        
        if sum([isinstance(i, int) for i in args]) == len(args):
            pass
        else:
            raise TypeError('integers are required')
            
        if len(args) == 1:
            start, stop, step = 0, args[0], 1
        elif len(args) == 2:
            start, stop, step = args[0], args[1], 1
        elif len(args) == 3:
            start, stop, step = args
        else:
            raise TypeError('range requires 1-3 int arguments')

        if step == 0:
            raise ValueError('range step cannot be a zero')

        self._start = start
        self._stop = stop
        self._step = step       
        self.current = self._start

    def __iter__(self):
        return self

    def __next__(self):
        while (self._stop-self.current)*self._step > 0:
            self.current += self._step
            return self.current - self._step
        raise StopIteration
        
            

numbers = custom_range(-10, 10, 2)
for i in numbers: 
    print(i)

-10
-8
-6
-4
-2
0
2
4
6
8


In [16]:
for i in range(-10, 10, 2): print(i)

-10
-8
-6
-4
-2
0
2
4
6
8


### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

In [17]:
class PrimaryKey:
    def __init__(self, name):
        self.name  = name       
    
    def __get__(self,instance, owner):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if value in instance.already_taken:
            raise ValueError('There is already such index in use')
        elif not isinstance(value, (str,int)):
            raise ValueError('Should be int or string')    
        else:
            instance.already_taken.add(value)
            instance.__dict__[self.name] = value
            
    def __del__(self, instance, index):
        instance.already_taken.remove(index)
        del instance.rows[index]        
        
            
class Table:
    """
    Class for table entity - it has name, set of indexes and dict of rows.
    """
    
    index = PrimaryKey('index')
    
    def __init__(self, table_name):
        self.table_name = table_name
        self.already_taken = set()        
        self.rows = {}
    
    def add_row(self, index, data):
        """
        Takes index and data as inputs, puts them in rows dict if index is free
        """
        self.index = index
        self.rows[index] = data
    
    def remove_row(self, index):        
        """
        Removes the row with the given index.
        """
        self.already_taken.remove(index)
        del self.rows[index] 
    
    def get_data(self):
        return self.rows
    
    def get_table_name(self):
        return self.table_name


In [18]:
a = Table('new_table')
a.get_data() ,a.already_taken

({}, set())

In [19]:
a.add_row('as', 'sd')
a.add_row('asa', 'sd')

In [20]:
a.already_taken

{'as', 'asa'}

In [21]:
a.get_data()

{'as': 'sd', 'asa': 'sd'}

In [22]:
a.add_row('asa', 'sdd')

ValueError: There is already such index in use

In [23]:
a.remove_row('as')
a.get_data()

{'asa': 'sd'}

In [24]:
a.add_row('as', 'sd')

In [25]:
 a.get_data()

{'asa': 'sd', 'as': 'sd'}

### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

In [26]:
class PositiveSmallIntegerField:
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if value < 0 or  value > 32767:
            raise ValueError('Should be between 0 and 32767')
        instance.__dict__[self.name] = value
    
    def __set_name__(self, owner, name):
        self.name = name

class PositiveSmallInteger:
    number = PositiveSmallIntegerField()

    def __init__(self, number):
        self.number = number
    
    def __add__(self, other):
        if isinstance(other, PositiveSmallInteger):
            return PositiveSmallInteger(self.number + other.number)
        else: 
            raise ValueError('Both should be PositiveSmallInteger')
                
    def __str__(self):
        return str(self.number)
    
    def __repr__(self):
        return str(self.number)

In [27]:
x = PositiveSmallInteger(10000)
y = PositiveSmallInteger(20000)

print(x, x+y)

10000 30000


In [28]:
PositiveSmallInteger(-23)

ValueError: Should be between 0 and 32767

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

Numpy здесь использую только для примера функции, которая будет выполняться внутри контекстного менеджера

In [29]:
import numpy as np
def fib(n):
        '''
        :param n: number of the Fibonacci number to return
        :return:  n-th Fibonacci number
        '''        
        def pow(x, n):  
            if n < 2: return x
            else:
                y = pow(x, n // 2)
                y = y.dot(y)
                if n % 2: y = x.dot(y)
                return y
        F = pow(np.matrix([[1,1],[1,0]], dtype=object), n)
        return np.array(F).flatten()[1]

In [30]:
from contextlib import contextmanager
import time

@contextmanager
def timing():
    """
    Returns time in seconds which operation took to execute
    """
    start_time = time.time()
    yield
    print(f"{time.time() - start_time:.3f} seconds")


with timing():
    x = [fib(i) for i in range(100)]
x[:10]

0.003 seconds


[1, 1, 1, 2, 3, 5, 8, 13, 21, 34]